In [6]:
import pandas as pd
import glob
import os
import numpy as np
import re
import time
from tqdm import tqdm
from konlpy.tag import Okt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from urllib.request import urlopen
import urllib.request as req
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from urllib.request import urlopen #url의 html 을 가져 오기 위한 패키지
from bs4 import BeautifulSoup  #크롤링 필수 패키지 설치하려면 cmd창에서 pip install bs4
import os
import re
from selenium import webdriver
from bs4 import BeautifulSoup #크롤링 도구
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import time
import json
import datetime
from konlpy.tag import Okt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pylab as pl
import statsmodels.api as sm
from sklearn.cluster import DBSCAN
import sklearn as skl
import sklearn.model_selection
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import random
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from ckonlpy.tag import Twitter
import string
import glob
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import warnings
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram
from matplotlib import font_manager, rc
from ckonlpy.tag import Postprocessor
from collections import Counter
import matplotlib
import tomotopy as tp
from pyvis.network import Network
#글씨체
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/H2HDRM.TTF').get_name()
rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus'] = False
#파일 한번에 불러와서 저장하기
flist = os.path.join('C:/Users/USER/Desktop/FW_ 고용기사 감성지수 Labeling 자료 공유', '*합본.csv') #끝에 합본.csv로 끝나는 것들 불러오기
colnames = ['date','a1','a2','title','a3','a4','text','user1','user2','user3','user4','user5','user6']
flist2 = os.path.join('C:/Users/USER/Desktop/FW_ 고용기사 감성지수 Labeling 자료 공유', '*.xlsx') #끝에 합본.csv로 끝나는 것들 불러오기
glob.glob(flist2)[0]  #파일 list 불러오기
pd.read_excel(glob.glob(flist2)[0], encoding='CP949',usecols=range(14))#,header=0, usecols=colnames)
def get_merge_csv(filelist):
    return pd.concat([pd.read_excel(i,encoding='CP949',usecols=range(13)) for i in filelist], axis=0)
def get_merge_csv2(filelist):
    return pd.concat([pd.read_csv(i,encoding='CP949',usecols=range(13)) for i in filelist], axis=0)
d = get_merge_csv(glob.glob(flist2))
d2 = get_merge_csv2(glob.glob(flist))
d.columns = colnames
d2.columns = colnames
d = pd.concat([d,d2])
d = d.reset_index(drop=True)
d.iloc[:,0:10]
d.columns = colnames

usecols=['date','title','text','user1','user2','user3','user4','user5','user6']
d = d[usecols] #필요한 컬럼만 받기

In [7]:
d

,date,title,text,user1,user2,user3,user4,user5,user6
0,20190101,인천시_ 노인 일자리 3만 2천개로 확대,인천시가 올해 920억 원을 들여 노인 일자리를 지난해보다 5천400개가 늘어난 3...,O,O,O,O,O,O
1,20190102,청년일자리 참여 사업장 모집,도는 2일부터 강원도형 청년일자리 참여 사업장 모집에 나선다. 이 사업은 인구 감소...,O,O,O,O,O,O
2,20190103,"주휴수당 피하려 알바쪼개기...""투잡 뛰어야 할판""",최저임금 쇼크에 아르바이트생 줄이는 자영업자 늘어 週 15시간 고용하면 주휴수당 줘...,O,O,O,O,O,O
3,20190105,12월 일자리 31만2천개 경기둔화 우려 속 10개월 최대(종합),실업률은 신규인력 유입으로 3.9%로 상승…임금도 전년비 3.2%↑ [AP=연합뉴스...,O,O,O,O,O,O
4,20190105,"시흥시 ""2022년까지 일자리 10만개 창출"" 대책본부 운영",민선 7기 일자리 종합계획 발표 시흥시 ⓒ News1 (시흥=뉴스1) 조정훈 기자 ...,O,O,O,O,O,O
...,...,...,...,...,...,...,...,...,...
49538,20181231,서부발전 7년간 9명 사망 모두 협력업체 소속,김용균씨 포함_ 작업 도중 사고 부상자 54명 … 협력업체가 52명 비정규직 근로자...,X,X,X,X,x,X
49539,20181231,내년 세계 경제 곳곳에 '지뢰' 5대 악재는?,"선진국 긴축 본격화…美금리인상·ECB 양적완화 종료 美-中 무역전쟁…""2019년은 ...",X,X,X,X,x,X
49540,20181231,주휴수당 안줘도 되는 `알바 쪼개기' 성행,주 15시간 이하 단기근로자 고용 업주들 최저임금 인상 편법 대응 태백에서 중화요리...,X,O,X,O,x,O
49541,20181231,사회적기업 종사자 대상 전세자금 최대90% 지원,연소득 5_000만원 이하인 사회적기업 또는 사회복지법인 및 시설 종사자에게 임차보...,X,X,X,X,x,X


In [8]:
for i in range(1,7): #라벨링한거 전처리 하기
    d._set_value(d['user'+str(i)] == 'X',
                 'user'+str(i),0)._set_value(d['user'+str(i)] == 'x','user'+str(i),0)._set_value(d['user'+str(i)] != 0,
                                                                                                 'user'+str(i),1)


In [10]:
d['sum'] = d[usecols[3:9]].sum(axis=1)#라벨링 합계 구하기
d['y'] = [1 if i > 3 else 0 for i in d['sum']] #1 과 0

In [11]:
d

,date,title,text,user1,user2,user3,user4,user5,user6,sum,y
0,20190101,인천시_ 노인 일자리 3만 2천개로 확대,인천시가 올해 920억 원을 들여 노인 일자리를 지난해보다 5천400개가 늘어난 3...,1,1,1,1,1,1,6.0,1
1,20190102,청년일자리 참여 사업장 모집,도는 2일부터 강원도형 청년일자리 참여 사업장 모집에 나선다. 이 사업은 인구 감소...,1,1,1,1,1,1,6.0,1
2,20190103,"주휴수당 피하려 알바쪼개기...""투잡 뛰어야 할판""",최저임금 쇼크에 아르바이트생 줄이는 자영업자 늘어 週 15시간 고용하면 주휴수당 줘...,1,1,1,1,1,1,6.0,1
3,20190105,12월 일자리 31만2천개 경기둔화 우려 속 10개월 최대(종합),실업률은 신규인력 유입으로 3.9%로 상승…임금도 전년비 3.2%↑ [AP=연합뉴스...,1,1,1,1,1,1,6.0,1
4,20190105,"시흥시 ""2022년까지 일자리 10만개 창출"" 대책본부 운영",민선 7기 일자리 종합계획 발표 시흥시 ⓒ News1 (시흥=뉴스1) 조정훈 기자 ...,1,1,1,1,1,1,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...
49538,20181231,서부발전 7년간 9명 사망 모두 협력업체 소속,김용균씨 포함_ 작업 도중 사고 부상자 54명 … 협력업체가 52명 비정규직 근로자...,0,0,0,0,0,0,0.0,0
49539,20181231,내년 세계 경제 곳곳에 '지뢰' 5대 악재는?,"선진국 긴축 본격화…美금리인상·ECB 양적완화 종료 美-中 무역전쟁…""2019년은 ...",0,0,0,0,0,0,0.0,0
49540,20181231,주휴수당 안줘도 되는 `알바 쪼개기' 성행,주 15시간 이하 단기근로자 고용 업주들 최저임금 인상 편법 대응 태백에서 중화요리...,0,1,0,1,0,1,3.0,0
49541,20181231,사회적기업 종사자 대상 전세자금 최대90% 지원,연소득 5_000만원 이하인 사회적기업 또는 사회복지법인 및 시설 종사자에게 임차보...,0,0,0,0,0,0,0.0,0


In [12]:
df = d[d['y'] == 1]
df

,date,title,text,user1,user2,user3,user4,user5,user6,sum,y
0,20190101,인천시_ 노인 일자리 3만 2천개로 확대,인천시가 올해 920억 원을 들여 노인 일자리를 지난해보다 5천400개가 늘어난 3...,1,1,1,1,1,1,6.0,1
1,20190102,청년일자리 참여 사업장 모집,도는 2일부터 강원도형 청년일자리 참여 사업장 모집에 나선다. 이 사업은 인구 감소...,1,1,1,1,1,1,6.0,1
2,20190103,"주휴수당 피하려 알바쪼개기...""투잡 뛰어야 할판""",최저임금 쇼크에 아르바이트생 줄이는 자영업자 늘어 週 15시간 고용하면 주휴수당 줘...,1,1,1,1,1,1,6.0,1
3,20190105,12월 일자리 31만2천개 경기둔화 우려 속 10개월 최대(종합),실업률은 신규인력 유입으로 3.9%로 상승…임금도 전년비 3.2%↑ [AP=연합뉴스...,1,1,1,1,1,1,6.0,1
4,20190105,"시흥시 ""2022년까지 일자리 10만개 창출"" 대책본부 운영",민선 7기 일자리 종합계획 발표 시흥시 ⓒ News1 (시흥=뉴스1) 조정훈 기자 ...,1,1,1,1,1,1,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...
49327,20181219,농림부_ 농촌일자리 8만7천 개 창출,동영상 뉴스 농림축산식품부는 2019년부터 4년간 농업과 농촌에 일자리 8만7천여 ...,1,0,0,1,1,1,4.0,1
49346,20181220,IT 스타트업_ 고용 창출 효자 역할 '톡톡',中企_ 내년 신규 채용규모 대폭 줄일 전망 IT 스타트업들_ 높은 고용 증가율 기록...,1,1,1,1,1,1,6.0,1
49349,20181220,외국인 노동자 62%가 월 200만원 이상 벌었다,최저임금 올라 고임금 늘었지만 경기침체로 1만명은 직장 잃어 올해부터 시행된 최저임...,1,1,1,1,1,0,5.0,1
49368,20181221,노조원 200만명 시대 친노조 정책 업고 작년 12만명 증가,근로자 100명 중 11명 꼴 노조원 공무원_ 노조조직률 69% 달해 문 정부 들어...,1,1,0,1,0,1,4.0,1


In [17]:
df = df.reset_index(drop=True)
#고용기사만 분류 완료

'/자료=잡코리아 올해 대학 졸업 예정자 중 11.6%만이 졸업 전에 정규직 취업에 성공한 것으로 조사됐다. 26일 잡코리아가 올해 국내 4년제 대학의 졸업 예정자 605명을 대상으로 ‘취업 현황’ 조사를 통해 ‘취업에 성공했는지’를 설문한 결과 ‘정규직으로 취업했다’는 답변이 11.6%로 나타났다. ‘비정규직으로 취업했다’고 밝힌 대학생은 13.2%였고 72.9%는 ‘아직 취업하지 못했다’고 답했다. 잡코리아에 따르면 올해 2월 기준 졸업 전 정규직 취업자 비율은 지난해 동일조사 대비 소폭 증가했다. 지난해 동일조사 결과 ‘정규직 취업자’는 11.0%였으나 올해는 11.6%로 소폭 증가세를 보인 것. ‘비정규직 취업자’는 지난해 10.0%에서 올해 13.2%로 3.2%포인트 늘어났다. 전공계열 별로는 ▲이공계열 전공자들의 정규직 취업률이 14.8%로 가장 높았다. 반면 ▲인문계열은 정규직 취업률이 7.3%에 그쳐 이공계열과 2배 가량 차이를 보였다. 성별로는 남학생과 여학생의 정규직 취업률이 각각 12.0%와 11.2%로 비슷했다. 정규직·비정규직 취업에 성공한 졸업예정자들은 중소기업(32.7%)에 가장 많이 취업했고 대기업(22.7%)과 중견기업(22.0%)에 취업했다는 답변이 이어졌다. 정규직 취업에 성공한 졸업예정자 중 64.3%가 ‘상반기에 신입직 구직활동을 계속하는 중’이라 답했다. 세부적으로 정규직 취업자 중 30.0%는 ‘상반기 대기업 신입공채를 준비한다’고 답했고 24.3%는 ‘공기업·공공기관 채용을 준비한다’고 답했다. 이어 비정규직 취업자는 대다수(83.8%)가 ‘상반기에 신입직 구직활동을 하는 중’인 것으로 나타났다. 이한듬 기자 *******@**.**.**'

In [34]:
table = pd.read_table('C:/Users/USER/Desktop/통합 문서1.txt',header = None, encoding = 'CP949')
table = table[:4807]
table

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,2.020110e+13,_,de0f80c6-cfd7-41e4-ade1-feb8c25b72b6,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,1,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,1제목,2020.11.01. 오전 2:22,_,내용서두,내용,NaN,NaN,2.020110e+13
1,2.020110e+13,_,c1d7e963-ce13-4a70-aec2-a1706ccd61f0,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,2,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,2제목,2020.11.01. 오전 2:22,_,내용서두,내용,NaN,NaN,2.020110e+13
2,2.020110e+13,_,795cf663-a6e8-43aa-8ecd-a77577cf11e7,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,3,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,3제목,2020.11.01. 오전 2:22,_,내용서두,내용,NaN,NaN,2.020110e+13
3,2.020110e+13,_,0038bf46-9896-4eb6-a830-110417237a16,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,4,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4제목,2020.11.01. 오전 2:22,_,내용서두,내용,NaN,NaN,2.020110e+13
4,2.020110e+13,_,ee656703-d555-4ac0-b9eb-c1c5965895c7,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,5,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,5제목,2020.11.01. 오전 2:22,_,내용서두,내용,NaN,NaN,2.020110e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4802,2.020110e+13,_,7e422f1c-58d9-465c-9329-fa22ced5f628,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,8,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4803제목,2020.11.01. 오전 2:22,_,내용서두,내용,NaN,NaN,2.020110e+13
4803,2.020110e+13,_,9d0915d9-0acb-4712-acf6-edd5c16b97b9,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,9,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4804제목,2020.11.01. 오전 2:22,_,내용서두,내용,NaN,NaN,2.020110e+13
4804,2.020110e+13,_,15cb3fe2-97a2-4714-8f8e-0c7be0b55618,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,10,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4805제목,2020.11.01. 오전 2:22,_,내용서두,내용,NaN,NaN,2.020110e+13
4805,2.020110e+13,_,1bea5639-712a-42e5-9c65-169afc5398a5,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,11,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4806제목,2020.11.01. 오전 2:22,_,내용서두,내용,NaN,NaN,2.020110e+13


In [37]:
table[17] = df['text']
table

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,2.020110e+13,_,de0f80c6-cfd7-41e4-ade1-feb8c25b72b6,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,1,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,1제목,2020.11.01. 오전 2:22,_,내용서두,인천시가 올해 920억 원을 들여 노인 일자리를 지난해보다 5천400개가 늘어난 3...,NaN,NaN,2.020110e+13
1,2.020110e+13,_,c1d7e963-ce13-4a70-aec2-a1706ccd61f0,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,2,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,2제목,2020.11.01. 오전 2:22,_,내용서두,도는 2일부터 강원도형 청년일자리 참여 사업장 모집에 나선다. 이 사업은 인구 감소...,NaN,NaN,2.020110e+13
2,2.020110e+13,_,795cf663-a6e8-43aa-8ecd-a77577cf11e7,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,3,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,3제목,2020.11.01. 오전 2:22,_,내용서두,최저임금 쇼크에 아르바이트생 줄이는 자영업자 늘어 週 15시간 고용하면 주휴수당 줘...,NaN,NaN,2.020110e+13
3,2.020110e+13,_,0038bf46-9896-4eb6-a830-110417237a16,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,4,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4제목,2020.11.01. 오전 2:22,_,내용서두,실업률은 신규인력 유입으로 3.9%로 상승…임금도 전년비 3.2%↑ [AP=연합뉴스...,NaN,NaN,2.020110e+13
4,2.020110e+13,_,ee656703-d555-4ac0-b9eb-c1c5965895c7,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,5,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,5제목,2020.11.01. 오전 2:22,_,내용서두,민선 7기 일자리 종합계획 발표 시흥시 ⓒ News1 (시흥=뉴스1) 조정훈 기자 ...,NaN,NaN,2.020110e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4802,2.020110e+13,_,7e422f1c-58d9-465c-9329-fa22ced5f628,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,8,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4803제목,2020.11.01. 오전 2:22,_,내용서두,동영상 뉴스 농림축산식품부는 2019년부터 4년간 농업과 농촌에 일자리 8만7천여 ...,NaN,NaN,2.020110e+13
4803,2.020110e+13,_,9d0915d9-0acb-4712-acf6-edd5c16b97b9,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,9,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4804제목,2020.11.01. 오전 2:22,_,내용서두,中企_ 내년 신규 채용규모 대폭 줄일 전망 IT 스타트업들_ 높은 고용 증가율 기록...,NaN,NaN,2.020110e+13
4804,2.020110e+13,_,15cb3fe2-97a2-4714-8f8e-0c7be0b55618,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,10,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4805제목,2020.11.01. 오전 2:22,_,내용서두,최저임금 올라 고임금 늘었지만 경기침체로 1만명은 직장 잃어 올해부터 시행된 최저임...,NaN,NaN,2.020110e+13
4805,2.020110e+13,_,1bea5639-712a-42e5-9c65-169afc5398a5,news.naver.com,NAVERNEWS,NaN,NaN,2.020110e+13,1,11,...,https://news.naver.com/main/read.nhn?mode=LSD&...,NaN,4806제목,2020.11.01. 오전 2:22,_,내용서두,근로자 100명 중 11명 꼴 노조원 공무원_ 노조조직률 69% 달해 문 정부 들어...,NaN,NaN,2.020110e+13


In [39]:
table.to_csv('C:/Users/USER/Desktop/문서123.txt',index = False, sep='\t')